# Dados e bibliotecas

obs: caminho das pastas seguindo ambiente kaggle do desafio escolhido

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')

In [14]:
# grupos do mesmo produto para análise de desempenho no treinamento
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['target'] = train.label_group.map(tmp)

# Funções

In [18]:
import string
import re #replace
 
def removePunctuation(text):
    punc_translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    return text.translate(punc_translator)

def removeMedidas(text):
    return re.sub(r'kg|cm|gr|ml|xl', "", text)


def removeNumer(text):
    return re.sub(r"^[\d\s]+|[\d][.\d]+|[\d]", "", text)

def removeSpecialCaracter(text):
    return re.sub(r"^[@.,\\\/\+\-\|\[\]]!+()", "", text)

def removeSpace(text):
    return " ".join(text.split())

In [19]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

In [ ]:
def formato_submissao(row):
    x = np.concatenate([row['predicao_tfidf'], row['predicao_hash']])
    return ' '.join( np.unique(x))

# TFIDF

Predição feita através da distância entre vetores das palavras

In [ ]:
# Limpeza do texto

train['title_clean'] = train['title'].str.lower()
train['title_clean'] = train['title_clean'].apply(removePunctuation)
train['title_clean'] = train['title_clean'].map(removeMedidas)
train['title_clean'] = train['title_clean'].apply(removeNumer)
train['title_clean'] = train['title_clean'].apply(removeSpecialCaracter)
train['title_clean'] = train['title_clean'].apply(removeNumer)
train['title_clean'] = train['title_clean'].apply(removeSpace)

In [ ]:
tfidf_vec = TfidfVectorizer(stop_words='english', 
                            binary=True, 
                            max_features=50000)
text_embeddings = tfidf_vec.fit_transform(train.title_clean).toarray()

Comparação de distância por lote por causa da memória 

In [ ]:
predicao_idf = []
CHUNK = 3500


CTS = len(train)//CHUNK
if len(train)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(train))
    
    # COSINE SIMILARITY DISTANCE
    cts = np.dot( text_embeddings, text_embeddings[a:b].T).T
    
    for k in range(b-a):
        IDX = np.where(cts[k,]>0.7)[0]
        filtrar_vizinhos = train.iloc[IDX].posting_id.values
        predicao_idf.append(filtrar_vizinhos)

In [ ]:
# adição ao df
train['predicao_tfidf'] = predicao_idf
train.head(5)

# Hash

Existem imagens com o mesmo hash na coluna image_hash

In [ ]:
tmp = train.groupby('image_phash').posting_id.agg('unique').to_dict()
train['predicao_hash'] = train.image_phash.map(tmp)
train.head()

# Avaliação do treinamento

In [ ]:
print("tfidf", train.apply(getMetric('predicao_tfidf'),axis=1).mean())

In [ ]:
print("hash", train.apply(getMetric('predicao_hash'),axis=1).mean())

# Aplicar classificação no dado de teste

In [39]:
test = pd.read_csv('/kaggle/input/shopee-product-matching/test.csv')

## TFIDF

In [ ]:
# limpeza do texto
test['title_clean'] = test['title'].str.lower()
test['title_clean'] = test['title_clean'].apply(removePunctuation)
test['title_clean'] = test['title_clean'].map(removeMedidas)
test['title_clean'] = test['title_clean'].apply(removeNumer)
test['title_clean'] = test['title_clean'].apply(removeSpecialCaracter)
test['title_clean'] = test['title_clean'].apply(removeNumer)
test['title_clean'] = test['title_clean'].apply(removeSpace)

In [ ]:
# tfidf
text_embeddings = tfidf_vec.transform(test.title_clean).toarray()

In [ ]:
predicao_idf = []
CHUNK = 3500


CTS = len(test)//CHUNK
if len(test)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(test))
    
    # COSINE SIMILARITY DISTANCE
    cts = np.dot( text_embeddings, text_embeddings[a:b].T).T
    
    for k in range(b-a):
        IDX = np.where(cts[k,]>0.7)[0]
        filtrar_vizinhos = test.iloc[IDX].posting_id.values
        predicao_idf.append(filtrar_vizinhos)

In [ ]:
# adição ao df
test['predicao_tfidf'] = predicao_idf
test.head(5)

## Hash

In [ ]:
# classificação por hash
tmp = test.groupby('image_phash').posting_id.agg('unique').to_dict()
test['predicao_hash'] = test.image_phash.map(tmp)
test.head()

# Formatação para arquivo de envio

Nesse tipo de competição, necessário importar o arquivo de teste e gerar o arquivo de submissão com as previsões no próprio código. Durante a submissão, o arquivo de teste é substituído com o verdadeiro conjunto de teste.

In [ ]:
test['matches'] = test.apply(formato_submissao, axis = 1)

In [ ]:
sample = pd.read_csv('/kaggle/input/shopee-product-matching/sample_submission.csv') 

In [ ]:
sample = test[['posting_id','matches']]

In [ ]:
sample.to_csv(f'/kaggle/working/submission.csv',mode='a',index=False,header=True)

sub = pd.read_csv('submission.csv')
sub.head(6)